In [120]:
"""
In this notebook, I try some pytorch basics,
changing standard tutorials.
Tried to understan what is pytorch returning at each step of execution.
"""

import torch
print(torch.__version__)

1.0.1


In [49]:
from numpy import vstack
from numpy import argmax
from pandas import read_csv
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score
from torch import Tensor
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torch.utils.data import random_split
from torch.nn import Linear
from torch.nn import ReLU
from torch.nn import Softmax
from torch.nn import Module
from torch.optim import SGD
from torch.nn import CrossEntropyLoss
from torch.nn.init import kaiming_uniform_
from torch.nn.init import xavier_uniform_

from torch import Tensor
from torch.nn import Linear
from torch.nn import ReLU
from torch.nn import Sigmoid
from torch.nn import Module
from torch.optim import SGD
	
from torch.optim import SGD
from torch.nn import BCELoss
from torch.nn.init import kaiming_uniform_
from torch.nn.init import xavier_uniform_

In [3]:
# dataset definition
class CSVDataset(Dataset):
    # load the dataset
    def __init__(self, path):
        # load the csv file as a dataframe
        df = read_csv(path, header=None)
        # store the inputs and outputs
        self.X = df.values[:, :-1]
        self.y = df.values[:, -1]
        # ensure input data is floats
        self.X = self.X.astype('float32')
        # label encode target and ensure the values are floats
        self.y = LabelEncoder().fit_transform(self.y)
 
    # number of rows in the dataset
    def __len__(self):
        return len(self.X)
 
    # get a row at an index
    def __getitem__(self, idx):
        return [self.X[idx], self.y[idx]]
 
    # get indexes for train and test rows
    def get_splits(self, n_test=0.33):
        # determine sizes
        test_size = round(n_test * len(self.X))
        train_size = len(self.X) - test_size
        # calculate the split
        return random_split(self, [train_size, test_size])

In [67]:
# prepare the data
path = 'https://raw.githubusercontent.com/jbrownlee/Datasets/master/ionosphere.csv'
dataset = CSVDataset(path)
# calculate split
train, test = dataset.get_splits()

In [68]:
train

In [69]:
df = read_csv(path, header=None)

In [70]:
df.head(10)

,0,1,2,3,4,5,6,7,8,9,...,25,26,27,28,29,30,31,32,33,34
0,1,0,0.99539,-0.05889,0.85243,0.02306,0.83398,-0.37708,1.00000,0.03760,...,-0.51171,0.41078,-0.46168,0.21266,-0.34090,0.42267,-0.54487,0.18641,-0.45300,g
1,1,0,1.00000,-0.18829,0.93035,-0.36156,-0.10868,-0.93597,1.00000,-0.04549,...,-0.26569,-0.20468,-0.18401,-0.19040,-0.11593,-0.16626,-0.06288,-0.13738,-0.02447,b
2,1,0,1.00000,-0.03365,1.00000,0.00485,1.00000,-0.12062,0.88965,0.01198,...,-0.40220,0.58984,-0.22145,0.43100,-0.17365,0.60436,-0.24180,0.56045,-0.38238,g
3,1,0,1.00000,-0.45161,1.00000,1.00000,0.71216,-1.00000,0.00000,0.00000,...,0.90695,0.51613,1.00000,1.00000,-0.20099,0.25682,1.00000,-0.32382,1.00000,b
4,1,0,1.00000,-0.02401,0.94140,0.06531,0.92106,-0.23255,0.77152,-0.16399,...,-0.65158,0.13290,-0.53206,0.02431,-0.62197,-0.05707,-0.59573,-0.04608,-0.65697,g
5,1,0,0.02337,-0.00592,-0.09924,-0.11949,-0.00763,-0.11824,0.14706,0.06637,...,-0.01535,-0.03240,0.09223,-0.07859,0.00732,0.00000,0.00000,-0.00039,0.12011,b
6,1,0,0.97588,-0.10602,0.94601,-0.20800,0.92806,-0.28350,0.85996,-0.27342,...,-0.81634,0.13659,-0.82510,0.04606,-0.82395,-0.04262,-0.81318,-0.13832,-0.80975,g
7,0,0,0.00000,0.00000,0.00000,0.00000,1.00000,-1.00000,0.00000,0.00000,...,1.00000,1.00000,1.00000,0.00000,0.00000,1.00000,1.00000,0.00000,0.00000,b
8,1,0,0.96355,-0.07198,1.00000,-0.14333,1.00000,-0.21313,1.00000,-0.36174,...,-0.65440,0.57577,-0.69712,0.25435,-0.63919,0.45114,-0.72779,0.38895,-0.73420,g
9,1,0,-0.01864,-0.08459,0.00000,0.00000,0.00000,0.00000,0.11470,-0.26810,...,-0.01326,0.20645,-0.02294,0.00000,0.00000,0.16595,0.24086,-0.08208,0.38065,b


In [71]:
import pandas as pd
pd.DataFrame(df.values[:, :-1]).head(5)
        

,0,1,2,3,4,5,6,7,8,9,...,24,25,26,27,28,29,30,31,32,33
0,1,0,0.99539,-0.05889,0.85243,0.02306,0.83398,-0.37708,1,0.0376,...,0.56811,-0.51171,0.41078,-0.46168,0.21266,-0.3409,0.42267,-0.54487,0.18641,-0.453
1,1,0,1,-0.18829,0.93035,-0.36156,-0.10868,-0.93597,1,-0.04549,...,-0.20332,-0.26569,-0.20468,-0.18401,-0.1904,-0.11593,-0.16626,-0.06288,-0.13738,-0.02447
2,1,0,1,-0.03365,1,0.00485,1,-0.12062,0.88965,0.01198,...,0.57528,-0.4022,0.58984,-0.22145,0.431,-0.17365,0.60436,-0.2418,0.56045,-0.38238
3,1,0,1,-0.45161,1,1,0.71216,-1,0,0,...,1,0.90695,0.51613,1,1,-0.20099,0.25682,1,-0.32382,1
4,1,0,1,-0.02401,0.9414,0.06531,0.92106,-0.23255,0.77152,-0.16399,...,0.03286,-0.65158,0.1329,-0.53206,0.02431,-0.62197,-0.05707,-0.59573,-0.04608,-0.65697


In [72]:
print(len(df.values))
pd.DataFrame(df.values[:, -1]).head(5)

351


,0
0,g
1,b
2,g
3,b
4,g


In [73]:
train

In [74]:
train_dl = DataLoader(train, batch_size=32, shuffle=True)

In [79]:
type(train_dl)

torch.utils.data.dataloader.DataLoader

In [80]:
for i, (inputs, targets) in enumerate(train_dl):
    print(i,type(inputs))
    print(inputs.numpy().shape)

0 <class 'torch.Tensor'>
(32, 34)
1 <class 'torch.Tensor'>
(32, 34)
2 <class 'torch.Tensor'>
(32, 34)
3 <class 'torch.Tensor'>
(32, 34)
4 <class 'torch.Tensor'>
(32, 34)
5 <class 'torch.Tensor'>
(32, 34)
6 <class 'torch.Tensor'>
(32, 34)
7 <class 'torch.Tensor'>
(11, 34)


In [81]:
train_dl

In [82]:
train_dl.dataset

In [83]:
train_dl.dataset[0]

[array([ 0.     ,  0.     ,  1.     , -1.     ,  1.     , -1.     ,
         1.     , -1.     ,  1.     , -1.     ,  1.     ,  1.     ,
         1.     ,  1.     ,  1.     , -1.     ,  1.     ,  1.     ,
         1.     ,  1.     ,  1.     ,  1.     ,  1.     , -1.     ,
         1.     , -1.     ,  1.     , -1.     ,  1.     ,  0.65625,
         0.     ,  0.     ,  1.     , -1.     ], dtype=float32), 0]

In [84]:
type( train_dl.dataset[0] )

list

In [85]:
print( len(train_dl.dataset))
train_dl.dataset[0][0].shape 

235


(34,)

In [86]:
train_dl.dataset[0][1].shape 

()

In [87]:
train_dl.dataset[0][1]

0

In [88]:
dataloader_iterator = iter(train_dl)
dlbatch_1 = next(dataloader_iterator)

In [91]:
dlbatch_1[0].shape

torch.Size([32, 34])

In [95]:
# model definition
class MLP(Module):
    # define model elements
    def __init__(self, n_inputs):
        super(MLP, self).__init__()
        # input to first hidden layer
        self.hidden1 = Linear(n_inputs, 10)
        kaiming_uniform_(self.hidden1.weight, nonlinearity='relu')
        self.act1 = ReLU()
        # second hidden layer
        self.hidden2 = Linear(10, 8)
        kaiming_uniform_(self.hidden2.weight, nonlinearity='relu')
        self.act2 = ReLU()
        # third hidden layer and output
        self.hidden3 = Linear(8, 1)
        xavier_uniform_(self.hidden3.weight)
        self.act3 = Sigmoid()
 
    # forward propagate input
    def forward(self, X):
        # input to first hidden layer
        X = self.hidden1(X)
        X = self.act1(X)
         # second hidden layer
        X = self.hidden2(X)
        X = self.act2(X)
        # third hidden layer and output
        X = self.hidden3(X)
        X = self.act3(X)
        return X

In [103]:
# define the optimization
model = MLP(34)
criterion = BCELoss()
optimizer = SGD(model.parameters(), lr=0.01, momentum=0.9)

In [104]:
inputs = dlbatch_1[0]
targets = dlbatch_1[1]
print("inputs",inputs.shape)
print("targets",targets.shape)

X = inputs 
y = targets

# label encode target and ensure the values are floats
y = LabelEncoder().fit_transform(y)
y = y.astype('float32')
y = y.reshape((len(y), 1))

print("X",X.shape)
print("y",y.shape)

inputs torch.Size([32, 34])
targets torch.Size([32])
X torch.Size([32, 34])
y (32, 1)


In [105]:
#1 step
# clear the gradients
optimizer.zero_grad()
# compute the model output
yhat = model(X)
y2 = torch.from_numpy(y)
# calculate loss
loss = criterion(yhat, y2)
# credit assignment
loss.backward()
# update model weights
optimizer.step()

In [117]:
loss

tensor(1.0095, grad_fn=<BinaryCrossEntropyBackward>)

In [118]:
#1more step
# clear the gradients
optimizer.zero_grad()
# compute the model output
yhat = model(X)
y2 = torch.from_numpy(y)
# calculate loss
loss = criterion(yhat, y2)
# credit assignment
loss.backward()
# update model weights
optimizer.step()

In [119]:
loss

tensor(0.9866, grad_fn=<BinaryCrossEntropyBackward>)